# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 80 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.05492


extracting tarball to tmp_2209.05492...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.05500


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.05492/ksdwarfs.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'definitions' from 'tmp_2209.05492/definitions.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.06022


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.05941/msarxiv.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2209.06022...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.06043


extracting tarball to tmp_2209.06043... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05593-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05593) | **GRAVITY faint: reducing noise sources in GRAVITY$^+$ with a fast  metrology attenuation system**  |
|| F. Widmann, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2022-09-14*|
|*Comments*| *Proceeding of SPIE Astronomical Telescopes + Instrumentation 2022*|
|**Abstract**| With the upgrade from GRAVITY to GRAVITY$^+$ the instrument will evolve into an all-sky interferometer that can observe faint targets, such as high redshift AGN. Observing the faintest targets requires reducing the noise sources in GRAVITY as much as possible. The dominant noise source, especially in the blue part of the spectrum, is the backscattering of the metrology laser light onto the detector. To reduce this noise we introduce two new metrology modes. With a combination of small hardware changes and software adaptations, we can dim the metrology laser during the observation without losing the phase referencing. For single beam targets, we can even turn off the metrology laser for the maximum SNR on the detector. These changes lead to an SNR improvement of over a factor of two averaged over the whole spectrum and up to a factor of eight in the part of the spectrum currently dominated by laser noise. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05814-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05814) | **The CARMENES search for exoplanets around M dwarfs: Stable  radial-velocity variations at the rotation period of AD~Leonis -- A test case  study of current limitations to treating stellar activity**  |
|| D. Kossakowski, et al. -- incl., <mark>M. Kürster</mark>, <mark>Th. Henning</mark>, <mark>T. Trifonov</mark>, <mark>M. Pérez-Torres</mark>, <mark>A. Pavlov</mark> |
|*Appeared on*| *2022-09-14*|
|*Comments*| *27 pages, 11 figures*|
|**Abstract**| Context: A challenge with radial-velocity (RV) data is disentangling the origin of signals either due to a planetary companion or to stellar activity. In fact, the existence of a planetary companion has been proposed, as well as contested, around the relatively bright, nearby M3.0V star AD Leo at the same period as the stellar rotation of 2.23d. Aims: We further investigate the nature of this signal. We introduce new CARMENES optical and near-IR RV data and an analysis in combination with archival data taken by HIRES and HARPS, along with more recent data from HARPS-N, GIANO-B, and HPF. Also, we address the confusion concerning the binarity of AD Leo. Methods: We consider possible correlations between the RVs and various stellar activity indicators accessible with CARMENES. We applied models within a Bayesian framework to determine whether a Keplerian model, a red-noise quasi-periodic model using a Gaussian process, or a mixed model would explain the observed data best. We also exclusively focus on spectral lines potentially associated with stellar activity. Results: The CARMENES RV data agree with the previously reported periodicity of 2.23d, correlate with some activity indicators, and exhibit chromaticity. However, when considering the entire RV data set, we find that a mixed model composed of a stable and a variable component performs best. Moreover, when recomputing the RVs using only spectral lines insensitive to activity, there appears to be some residual power at the period of interest. We therefore conclude that it is not possible to determinedly prove that there is no planet orbiting in synchronization with the stellar rotation given our data, current tools, machinery, and knowledge of how stellar activity affects RVs. We do rule out planets more massive than 27M_E (=0.084M_J). We also exclude any binary companion around AD Leo with Msini > 3-6M_J on orbital periods <14yr. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05492-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05492) | **Ultra-Diffuse Galaxies as Extreme Star-forming Environments I: Mapping  Star Formation in HI-Rich UDGs**  |
|| Erin Kado-Fong, <mark>Jenny E. Greene</mark>, Song Huang, <mark>Andy Goulding</mark> |
|*Appeared on*| *2022-09-14*|
|*Comments*| *Submitted to ApJ, 27 pages, 17 figures*|
|**Abstract**| Ultra-Diffuse Galaxies are both extreme products of galaxy evolution and extreme environments in which to test our understanding of star formation. In this work, we contrast the spatially resolved star formation activity of a sample of 22 HI-selected UDGs and 35 low-mass galaxies from the NASA Sloan Atlas (NSA) within 120 Mpc. We employ a new joint SED fitting method to compute star formation rate and stellar mass surface density maps that leverage the high spatial resolution optical imaging data of the Hyper Suprime-Cam Subaru Strategic Program (HSC-SSP) and the UV coverage of GALEX, along with HI radial profiles estimated from a subset of galaxies that have spatially resolved HI maps. We find that the UDGs have low star formation efficiencies as a function of their atomic gas down to scales of 500 pc. We additionally find that the stellar mass-weighted sizes of our UDG sample are unremarkable when considered as a function of their HI mass -- their stellar sizes are comparable to the NSA dwarfs at fixed HI mass. This is a natural result in the picture where UDGs are forming stars normally, but at low efficiencies. We compare our results to predictions from contemporary models of galaxy formation, and find in particular that our observations are difficult to reproduce in models where UDGs undergo stellar expansion due to vigorous star formation feedback should bursty star formation be required down to $z=0$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05941-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05941) | **The Active Chromospheres of Lithium-Rich Red Giant Stars**  |
|| Christopher Sneden, et al. -- incl., <mark>Anohita Mallick</mark>, <mark>Suvrath Mahadevan</mark> |
|*Appeared on*| *2022-09-14*|
|*Comments*| *Astrophysical Journal, in press*|
|**Abstract**| We have gathered near-infrared $zyJ$-band high resolution spectra of nearly 300 field red giant stars with known lithium abundances in order to survey their \species{He}{i} $\lambda$10830 absorption strengths. This transition is an indicator of chromospheric activity and/or mass loss in red giants. The majority of stars in our sample reside in the red clump or red horizontal branch based on their $V-J,M_V$ color-magnitude diagram and their Gaia \teff, \logg\ values. Most of our target stars are Li-poor in the sense of having normally low Li abundances, defined here as \eps{Li}~$<$~1.25. Over 90\% of these Li-poor stars have weak $\lambda$10830 features. But more than half of the 83 Li-rich stars (\eps{Li}~$>$~1.25) have strong $\lambda$10830 absorptions. These large $\lambda$10830 lines signal excess chromospheric activity in Li-rich stars; there is almost no indication of significant mass loss. The Li-rich giants also may have a higher binary fraction than do Li-poor stars, based on their astrometric data. It appears likely that both residence on the horizontal branch and present or past binary interaction play roles in the significant Li-He connection established in this survey. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06022-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06022) | **The BlueMUSE data reduction pipeline: lessons learned from MUSE and  first design choices**  |
|| Peter M. Weilbacher, et al. -- incl., <mark>Martin M. Roth</mark> |
|*Appeared on*| *2022-09-14*|
|*Comments*| *10 pages, 4 figures. Presented at SPIE "Astronomical Telescopes and Instrumentation", July 2022, in Montr\'eal, as part of "Software and Cyberinfrastructure for Astronomy VII"*|
|**Abstract**| BlueMUSE is an integral field spectrograph in an early development stage for the ESO VLT. For our design of the data reduction software for this instrument, we are first reviewing capabilities and issues of the pipeline of the existing MUSE instrument. MUSE has been in operation at the VLT since 2014 and led to discoveries published in more than 600 refereed scientific papers. While BlueMUSE and MUSE have many common properties we briefly point out a few key differences between both instruments. We outline a first version of the flowchart for the science reduction, and discuss the necessary changes due to the blue wavelength range covered by BlueMUSE. We also detail specific new features, for example, how the pipeline and subsequent analysis will benefit from improved handling of the data covariance, and a more integrated approach to the line-spread function, as well as improvements regarding the wavelength calibration which is of extra importance in the blue optical range. We finally discuss how simulations of BlueMUSE datacubes are being implemented and how they will be used to prepare the science of the instrument. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05541-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05541) | **Towards a multi-tracer timeline of star formation in the LMC -- II. The  formation and destruction of molecular clouds**  |
|| Jacob L. Ward, et al. -- incl., <mark>J. M. Diederik Kruijssen</mark> |
|*Appeared on*| *2022-09-14*|
|*Comments*| *18 pages (including appendices), 10 figures, 2 tables; MNRAS in press (accepted August 15, 2022). Figures 3, 4, 5, and 7 show the main results of the paper*|
|**Abstract**| The time-scales associated with various stages of the star formation process represent major unknowns in our understanding of galactic evolution, as well as of star and planet formation. This is the second paper in a series aiming to establish a multi-tracer time-line of star formation in the Large Magellanic Cloud (LMC), focusing on the lifecycle of molecular clouds. We use a statistical method to determine a molecular cloud lifetime in the LMC of $t_{\text{CO}}=11.8^{+2.7}_{-2.2}$ Myr. This short time-scale is similar to the cloud dynamical time, and suggests that molecular clouds in the LMC are largely decoupled from the effects of galactic dynamics and have lifetimes set by internal processes. This provides a clear contrast to atomic clouds in the LMC, of which the lifetimes are correlated with galactic dynamical time-scales. We additionally derive the time-scale for which molecular clouds and HII regions co-exist as $t_{\text{fb}}=1.2^{+0.3}_{-0.2}$ Myr, implying an average feedback front expansion velocity of 12 km s$^{-1}$, consistent with expansion velocities of HII regions in the LMC observed directly using optical spectroscopy. Taken together, these results imply that the molecular cloud lifecycle in the LMC proceeds rapidly and is regulated by internal dynamics and stellar feedback. We conclude by discussing our measurements in the context of previous work in the literature, which reported considerably longer lifetimes for molecular clouds in the LMC, and find that these previous findings resulted from a subjective choice in timeline calibration that is avoided by our statistical methodology. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06043-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06043) | **Investigating the impact of different velocity fields on the spectral  appearance of Wolf-Rayet stars**  |
|| Roel R. Lefever, et al. -- incl., <mark>Helge Todt</mark> |
|*Appeared on*| *2022-09-14*|
|*Comments*| *2 pages, 1 figure, to be published in the Proceedings of the International Astronomical Union for the IAU Symposium 361 "Massive Stars Near and Far"*|
|**Abstract**| The emission line spectra of WR stars are often formed completely in the optically thick stellar wind. Hence, any assumption on the wind velocity law in a spectral analysis has a profound impact on the determination of the stellar parameters. By comparing Potsdam Wolf-Rayet (PoWR) model spectra calculated with different $\beta$ laws, we show that the velocity field heavily influences the spectra: by using the appropriate $\beta$ laws, the entire range of late and early types can be covered with the same stellar model. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05910-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05910) | **The Climate and Compositional Variation of the Highly Eccentric Planet  HD 80606 b]{The Climate and Compositional Variation of the Highly Eccentric  Planet HD 80606 b -- the rise and fall of carbon monoxide and elemental  sulfur**  |
|| Shang-Min Tsai, et al. -- incl., <mark>Maria Steinrueck</mark>, <mark>Nikole Lewis</mark> |
|*Appeared on*| *2022-09-14*|
|*Comments*| *18 pages, 18 figures, submitted to MNRAS, comments welcome*|
|**Abstract**| The gas giant HD 80606 b has a highly eccentric orbit (e $\sim$ 0.93). The variation due to the rapid shift of stellar irradiation provides a unique opportunity to probe the physical and chemical timescales and to study the interplay between climate dynamics and atmospheric chemistry. In this work, we present integrated models to study the atmospheric responses and the underlying physical and chemical mechanisms of HD 80606 b. We first run three-dimensional general circulation models (GCMs) to establish the atmospheric thermal and dynamical structures for different atmospheric metallicities and internal heat. Based on the GCM output, we then adopted a 1D time-dependent photochemical model to investigate the compositional variation along the eccentric orbit. The transition of the circulation patterns of HD 80606 b matched the dynamics regimes in previous works. Our photochemical models show that efficient vertical mixing leads to deep quench levels of the major carbon and nitrogen species and the quenching behavior does not change throughout the eccentric orbit. Instead, photolysis is the main driver of the time-dependent chemistry. A transient state of [CO]/[CH$_4$] $>$ 1 after periastron is confirmed for all metallicity and internal heat cases. The upcoming JWST Cycle 1 GO program will be able to track this real-time CH$_4$--CO conversion and infer the chemical timescale. Furthermore, sulfur species initiated by sudden heating and photochemical forcing exhibit both short-term and long-term cycles, opening an interesting avenue for detecting sulfur on exoplanets. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure figs/HD806-orbit.pdf</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.05593.md
    + _build/html/tmp_2209.05593/./figures/metrology_detector.png
    + _build/html/tmp_2209.05593/./figures/gfaint_pos.png
    + _build/html/tmp_2209.05593/./figures/intensity_dark.png
exported in  _build/html/2209.05814.md
    + _build/html/tmp_2209.05814/./20220331_adleo_activity_carmvis1carmvis2_correlations.png
    + _build/html/tmp_2209.05814/./20220402_adleo_periodogram_carmvis_activity.png
    + _build/html/tmp_2209.05814/./harps123456vis1234niryh_theory_20220405.png
    + _build/html/tmp_2209.05814/./allrv_timeseries_20220707.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\GP}{GRAVITY^+}$
$\newcommand{\baselinestretch}{1.0}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\GP$}{GRAVITY^+}$
$\newcommand{$\baselinestretch$}{1.0}$</div>



<div id="title">

#  with a fast metrology attenuation system

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.05593-b31b1b.svg)](https://arxiv.org/abs/2209.05593)<mark>Appeared on: 2022-09-14</mark> - _Proceeding of SPIE Astronomical Telescopes + Instrumentation 2022_

</div>
<div id="authors">

F. Widmann, et al. -- incl., <mark><mark>L. Kreidberg</mark></mark>

</div>
<div id="abstract">

**Abstract:** With the upgrade from GRAVITY to GRAVITY+ the instrument will evolve to an all-sky interferometer that can observe faint targets, such as high redshift AGN. Observing the faintest targets requires reducing the noise sources in GRAVITY as much as possible. The dominant noise source, especially in the blue part of the spectrum, is the backscattering of the metrology laser light onto the detector. To reduce this noise we introduce two new metrology modes. With a combination of small hardware changes and software adaptations, we can dim the metrology laser during the observation without losing the phase referencing. For single beam targets, we can even turn off the metrology laser for the maximum SNR on the detector. These changes lead to a SNR improvement of over a factor of two averaged over the whole spectrum and up to a factor of eight in the part of the spectrum currently dominated by laser noise.

</div>

<div id="div_fig1">

<img src="tmp_2209.05593/./figures/metrology_detector.png" alt="Fig3" width="100%"/>

**Figure 3. -** Cut out from the detector image of a dark frame in all three modes. Top: metrology on, where clear backscattering can be seen in left part of the spectra. Middle: metrology faint, with only a small peak from the carrier laser remaining. Bottom: metrology off, no spectra visible anymore. The white rectangle in all plots shows a typical extraction region for a spectrum. (*fig:met_det*)

</div>
<div id="div_fig2">

<img src="tmp_2209.05593/./figures/gfaint_pos.png" alt="Fig7" width="100%"/>

**Figure 7. -** First test of an astrometric measurement with GRAVITY-faint. The left plots show the visibility phase over 4 exposures of \SI{2}{\min} each, with the average phase as a bold line and the phase per DIT shaded out. The right plot shows the astrometry of the four files, relative to the calibrated phase center. (*fig:gfaint_pos*)

</div>
<div id="div_fig3">

<img src="tmp_2209.05593/./figures/intensity_dark.png" alt="Fig2" width="100%"/>

**Figure 2. -** Measured intensity on a \SI{10}{\second} dark frame in medium resolution with an indication for the dominant sources over the spectrum. (*fig:intensity*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\juliet}{\texttt{juliet}}$
$\newcommand{\serval}{\texttt{serval}}$
$\newcommand{\george}{\texttt{george}}$
$\newcommand{\exostriker}{\texttt{Exo-Striker}}$
$\newcommand{\tess}{TESS}$
$\newcommand{\gaia}{\textit{Gaia}}$
$\newcommand{\jwst}{JWST}$
$\newcommand{\au}{au}$
$\newcommand{\XX}{\textbf{\textcolor{red}{XX}}}$
$\newcommand{\addref}[1]{(add ref)}$
$\newcommand{\person}[2]{\textcolor{magenta}{\bf @#1: #2}}$
$\newcommand{\TODO}[1]{\textcolor{magenta}{\textsc{todo:} \textit{#1}}}$
$\newcommand{\CHANGE}[1]{\textcolor{red}{\textbf{#1}}}$
$\newcommand$
$\newcommand{\abs}[1]{|#1|}$
$\newcommand{\ms}{m s^{-1}}$
$\newcommand{\mearth}{M_\oplus}$
$\newcommand{\rearth}{R_\oplus}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\juliet$}{\texttt{juliet}}$
$\newcommand{$\serval$}{\texttt{serval}}$
$\newcommand{$\george$}{\texttt{george}}$
$\newcommand{$\exostriker$}{\texttt{Exo-Striker}}$
$\newcommand{$\tess$}{TESS}$
$\newcommand{$\gaia$}{\textit{Gaia}}$
$\newcommand{$\jwst$}{JWST}$
$\newcommand{$\au$}{au}$
$\newcommand{$\XX$}{\textbf{\textcolor{red}{XX}}}$
$\newcommand{$\addref$}[1]{(add ref)}$
$\newcommand{$\person$}[2]{\textcolor{magenta}{\bf @#1: #2}}$
$\newcommand{$\TODO$}[1]{\textcolor{magenta}{$\textsc${todo:} \textit{#1}}}$
$\newcommand{$\CHANGE$}[1]{\textcolor{red}{\textbf{#1}}}$
$\newcommand$
$\newcommand{$\abs$}[1]{|#1|}$
$\newcommand{$\ms$}{m s^{-1}}$
$\newcommand{$\mearth$}{M_\oplus}$
$\newcommand{$\rearth$}{R_\oplus}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.05814-b31b1b.svg)](https://arxiv.org/abs/2209.05814)<mark>Appeared on: 2022-09-14</mark> - _27 pages, 11 figures_

</div>
<div id="authors">

D. Kossakowski, et al. -- incl., <mark><mark>M. Kürster</mark></mark>, <mark><mark>Th. Henning</mark></mark>, <mark><mark>T. Trifonov</mark></mark>, <mark><mark>M. Pérez-Torres</mark></mark>, <mark><mark>A. Pavlov</mark></mark>

</div>
<div id="abstract">

**Abstract:** A challenge with radial-velocity (RV) data is disentangling the origin of signals either due to a planetary companion or to stellar activity. In fact, the existence of a planetary companion has been proposed, as well as contested, around the relatively bright, nearby M3.0 V star AD Leo at the same period as the stellar rotation of 2.23 d.We further investigate the nature of this signal. We introduce new CARMENES optical and near-IR RV data and an analysis in combination with archival data taken by HIRES and HARPS, along with more recent data from HARPS-N, GIANO-B, and HPF. Additionally, we address the confusion concerning the binarity of AD Leo.We consider possible correlations between the RVs and various stellar activity indicators accessible with CARMENES. We additionally applied models within a Bayesian framework to determine whether a Keplerian model, a red-noise quasi-periodic model using a Gaussian process, or a mixed model would explain the observed data best. We also exclusively focus on spectral lines potentially associated with stellar activity.The CARMENES RV data agree with the previously reported periodicity of 2.23 d, correlate with some activity indicators, and exhibit chromaticity.  However, when considering the entire RV data set, we find that a mixed model composed of a stable and a variable component performs best. Moreover, when recomputing the RVs using only spectral lines insensitive to activity, there appears to be some residual power at the period of interest. We therefore conclude that it is not possible to determinedly prove that there is no planet orbiting in synchronization with the stellar rotation given our data, current tools, machinery, and knowledge of how stellar activity affects RVs. We do rule out planets more massive than 27 M$_{\oplus}$(= 0.084 M$_\textnormal{Jup}$). Likewise, we exclude any binary companion around AD Leo with M$\sin{i}$greater than 3--6 M$_\textnormal{Jup}$on orbital periods$<14$yr.

</div>

<div id="div_fig1">

<img src="tmp_2209.05814/./20220331_adleo_activity_carmvis1carmvis2_correlations.png" alt="Fig11.1" width="50%"/><img src="tmp_2209.05814/./20220402_adleo_periodogram_carmvis_activity.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Correlation plots with the RVs (\textit{left}) and GLS periodograms (\textit{right}) of the various stellar activity indicators from the CARMENES VIS spectroscopic data for AD Leo. For the correlation plots, the circles and triangles represent the first and second subset of CARMENES VIS data, respectively. Data points are color-coded with the rotation phase. The Pearson-$r$ correlation coefficient combining both subsets is shown within each panel (R). For the periodograms, only the first season of the CARMENES VIS data was considered for plotting. The orange vertical solid and dashed lines represent the rotation period at $P=2.23$ d and its daily alias at 1.81 d. The horizontal dotted, dot-dashed, and dashed blue lines represent the 10 \%, 1 \%, and 0.1 \% FAP levels. (*fig:carmvisactivity*)

</div>
<div id="div_fig2">

<img src="tmp_2209.05814/./harps123456vis1234niryh_theory_20220405.png" alt="Fig3" width="100%"/>

**Figure 3. -** Radial-velocity semi-amplitudes as a function of wavelength for the wavelength chunks from HARPS and the CARMENES VIS and NIR spectographs. The gray horizontal lines for each data point correspond to the wavelength coverage considered when recomputing the RV for the wavelength chunk. The green dots connected by a dashed line represent the theoretical values of a 1.5\% spot coverage on a 3700 K, $v \sin{i}$ of 5 k$\ms$ star with a temperature difference of 200 K taken from \cite{Reiners2010}. The theoretical values are binned in 2 to serve as a better comparison to the wavelength bins provided by the real data. (*fig:Kvswavelength*)

</div>
<div id="div_fig3">

<img src="tmp_2209.05814/./allrv_timeseries_20220707.png" alt="Fig7" width="100%"/>

**Figure 7. -** Time series of all RVs with instrumental offsets accounted for. The RV uncertainties are included, though many are too small to be seen in the plots. The HIRES and HARPS data span a large time range and the rest of the data come in $\sim$12 years later after the time when the majority of the previous data were taken. The time axis is interrupted in several places, and stretched differently between the individual sections. Additionally, the majority of HIRES and HARPS time series are overlapping each other. Four of the HARPS-N data overlap with CARMENES data; and the GIANO-B data are taken all within the first observing run for HARPS-N. The first season of HPF data overlaps with the HARPS-N and GIANO-B data sets whereas the HPF second season overlaps with the HARPS-N and CARMENES second season. (*fig:allrv_timeseries*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

32  publications in the last 7 days.
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers